In [1]:
# Here we are importing the skimage package as this is where the edge detection
# program is.
import skimage
import skimage.feature
import skimage.viewer

# Then importing a couple other things. The image processing is being handled by
# Pillow.
import numpy as np
from PIL import Image
import os
import PIL.ImageOps

# Lastly here are a couple paths that we will be using in order to save correctly
path = "Collected_Images/Split_Images/"
formation_path = "Collected_Images/Edge_Detection/Formation_Imgs/"
out_path = "Collected_Images/Edge_Detection/"

# Toggle this if you want to save off the pictures that are created along the way
delete = True

C:\Users\lawso\AppData\Local\Continuum\anaconda3\envs\geospatial\lib\site-packages\skimage\viewer\utils\__init__.py:1: UserWarning: Recommended matplotlib backend is `Agg` for full skimage.viewer functionality.
  from .core import *


In [2]:
'''
    Given an image, the function will crop the image so that only the formation
    part of the image is saved.
    
    :param path: Path that the original image is in
    :param i: The name of the image (with the extension) that we will use
    :param name: Name of the image
    :param save_path: Where the image will be saved to

'''

def extract_formation_img(path, i, name, save_path):
    img = Image.open(path + i)
    
    w, h = img.size
    
    left = 80
    top = 0
    right = w
    bottom = h
    
    img = img.crop((left,
                    top,
                    right,
                    bottom))
    img.save(save_path + name + "-formation.png")

In [3]:
'''
    Given an image and other values, the function will save off an image that
    was returned from the edge detection program and a color inverted version
    of the edge detection program
    
    :param img_path: name of the image with extension 
    :param s: sigma value
    :param l: low_threshold
    :param h: high_threshold
    :param delete: delete the pictures generated to create the final image
    
'''

def edge_detect(img_path, s, l, h, delete, save_path):
    cur_img = skimage.io.imread(fname = out_path + img_path)
    
    if(len(cur_img.shape) > 2):
        cur_img = cur_img[:,:,0]
    
    cur_img = cur_img * (1.0/255)
    
    edges = skimage.feature.canny(
        image=cur_img,
        sigma=s,
        low_threshold=l,
        high_threshold=h,
    )
    
    skimage.io.imsave(arr = edges, fname = out_path + "black-white.png")
    thing = Image.open(save_path + "black-white.png")
    invert = PIL.ImageOps.invert(thing)
    invert.save(save_path + "outlined.png")
    
    w_1, h_1 = thing.size
    w_2, h_2 = invert.size
    
    cur_comb = Image.new('RGB', (w_1+w_2, h_1))
    cur_comb.paste(thing, (0,0))
    cur_comb.paste(invert, (w_1,0))
    
    cur_comb.save(save_path + "black-white-edge.png")
    
    if(delete):
        os.remove(save_path + "outlined.png")
        os.remove(save_path + "black-white.png")
        os.remove(save_path + "black-white-edge.png")
        
    return cur_comb

In [4]:
'''
    This process will take an image and strip off the necessary RGB layer,
    run it through the edge detection method, and stitch the results together
    as one big image to show the changes throughout the program. The images in
    order are:
    1st: Original image
    2nd: Image stripped to one of it's RGB layers
    3rd: Stripped image after going through edge detection
    4th: The 3rd image, but color inverted
    
    :param form_path: Path to the formation image
    :param form_img: The name of the formation image with extension
    :param img_name: The name of the image to save
    :param delete: Delete the pictures constructed throughout the program
    :param save_path: If delete is false, this is where the images will be saved to
    :param pro_path: Where to save the final images
    :param color: Which RGB layer to use, 0 - Red, 1 - Green, 2 - Blue
    

'''
def process_1(form_path, form_img, img_name, delete, save_path, pro_path, color):
    image = skimage.io.imread(fname = form_path + form_img)
    temp = np.copy(image)
    x = temp
    result = x[:, :, color]
    result = result * (1.0/255)
    
    skimage.io.imsave(arr = result, fname = save_path + "reduced-dimension.png")

    sigma = 0
    low_threshold = 0.7
    high_threshold = 1


    edge_img = edge_detect("reduced-dimension.png", 
                           sigma, 
                           low_threshold, 
                           high_threshold, 
                           delete, 
                           out_path
                          )

    img_1 = Image.open(form_path + form_img)
    img_2 = Image.open(save_path + "reduced-dimension.png")

    w1, h1 = img_1.size
    w2, h2 = img_2.size
    w3, h3 = edge_img.size

    combine = Image.new('RGB', (w1+w2+w3, h1))
    combine.paste(img_1, (0,0))
    combine.paste(img_2, (w1,0))
    combine.paste(edge_img, (w1+w2,0))

    combine.save(pro_path + "Process_1_" + img_name + ".png")

    if(delete):
        os.remove(save_path + "reduced-dimension.png")

In [5]:
# Here we are going through all the constructed images created in the other
# Jupyter notebook and we are splitting them off by which color we use.
# This being Red, Green and Blue

file_list = os.listdir(path)
paths = ["Collected_Images/Edge_Detection/Process_1_Red/",
         "Collected_Images/Edge_Detection/Process_1_Green/",
         "Collected_Images/Edge_Detection/Process_1_Blue/"
        ]

for i in range(0, len(file_list)):
    cur_img = file_list[i]
    cur_name = cur_img[0: -4]
    
    extract_formation_img(path,
                          cur_img,
                          cur_name,
                          formation_path
                         )
    cur_form_name = cur_name + "-formation.png"
    
    for j in range(0, 3):
        process_1(formation_path,
                  cur_form_name,
                  cur_name,
                  delete,
                  out_path,
                  paths[j],
                  j
                 )

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\lawso\AppData\Local\Continuum\anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Collected_Images/Edge_Detection/black-white.png is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  app.launch_new_instance()
C:\Users\lawso\AppData\Local\Continuum\anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Collected_Images/Edge_Detection/black-white.png is a low contrast image
  app.launch_new_instance()
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\lawso\AppData\Local\Continuum\anaconda3\envs\geospatial\lib\site-packages\ipykernel_laun

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

...

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0